In [1]:
%reload_ext autoreload
%matplotlib inline

In [2]:
%autoreload 2

import sys,os,glob
import psutil

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
from pprint import pprint
import datetime as dt


from matplotlib.colors import LogNorm
from matplotlib.gridspec import GridSpec
import cartopy.crs as ccrs

import datetime as dt
import re
import gc
import matplotlib.animation as animation
import warnings
import pickle
from scipy.interpolate import interp1d


#-- Own settings

from settings import *

## current script object
thismodule = sys.modules[__name__]
workdir = os.getcwd()
# Add own module library to path
moduledir, fcndir = defineDir(workdir)

# load own libraries
from conditionalstats import *
from conditionalstats_chunked import *
from plot1D import *

# own DYAMOND functions
from fcns_load_DYAMOND_SAM import *
# own TOOCAN functions
from load_TOOCAN_DYAMOND_modif_BF import load_TOOCAN_DYAMOND

Own modules available: ['conditionalstats', 'plot2D', 'plotCondPDFs', 'plot1D', 'conditionalstats_chunked']
Own modules available: ['fcns_load_DYAMOND_SAM', 'load_TOOCAN_DYAMOND_modif', 'load_TOOCAN_DYAMOND', 'load_TOOCAN_DYAMOND_modif_BF', 'casestudy']


In [3]:
path = '/data/fiolleau/DYAMOND/TOOCAN/TOOCAN_v2.07/GLOBAL/2016/FileTracking'

In [4]:
filename = 'TOOCAN-GLOBAL-20160801-20160831.dat.gz'

In [6]:
toocan = load_TOOCAN_DYAMOND(os.path.join(path,filename))

/data/fiolleau/DYAMOND/TOOCAN/TOOCAN_v2.07/GLOBAL/2016/FileTracking/TOOCAN-GLOBAL-20160801-20160831.dat.gz


In [7]:
toocan[0]

< MCS_IntParameters object:
 . label: 1
 . qc_MCS: 11100
 . duration: 12.5
 . classif: 2
 . Tmax: 0.0
 . Utime_Init: 17014.18
 . lonInit: 115.04
 . latInit: 20.71
 . Utime_End: 17014.42
 . lonEnd: 116.37
 . latEnd: 21.88
 . lonmin: 114.93
 . latmin: 20.71
 . lonmax: 116.37
 . latmax: 21.89
 . vavg: 4.2
 . dist: 189.2
 . olrmin: 188.0
 . surfmaxPix_172Wm2: 0
 . surfmaxkm2_172Wm2: 32864.0
 . surfmaxkm2_132Wm2: 32592.0
 . surfmaxkm2_110Wm2: 32320.0
 . surfmaxkm2_90Wm2: 31520.0
 . surfcumkm2_172Wm2: 354480.0
 . surfcumkm2_132Wm2: 0
 . surfcumkm2_110Wm2: 0
 . surfcumkm2_90Wm2: 0
 . precip_total: 0
 . precip_max: 0
 . maxSurf00mmh_km2: 0
 . maxSurf02mmh_km2: 0
 . maxSurf05mmh_km2: 0
 . maxSurf10mmh_km2: 0
 . classif_JIRAK: 0.0
 . localtime_Init: 17014.6737
 . localtime_End: 17015.1774
 . clusters: <class 'load_TOOCAN_DYAMOND_modif_BF.MCS_Lifecycle'>
 >

In [8]:
N_mcs = len(toocan)
print(N_mcs,'TOOCAN objects')

276179 TOOCAN objects


In [9]:
date_ref = dt.datetime(year=1970,month=1,day=1)
times_init = np.array([date_ref+dt.timedelta(days=toocan[i_mcs].Utime_Init) for i_mcs in range(N_mcs)])

In [10]:
times_init

array([datetime.datetime(2016, 8, 1, 4, 19, 12),
       datetime.datetime(2016, 8, 1, 8, 52, 48),
       datetime.datetime(2016, 8, 1, 0, 43, 12), ...,
       datetime.datetime(2016, 8, 31, 11, 16, 48),
       datetime.datetime(2016, 8, 31, 10, 48),
       datetime.datetime(2016, 8, 31, 11, 31, 12)], dtype=object)

In [11]:
# Tmax

In [12]:
Tmax = np.array([toocan[i_mcs].Tmax for i_mcs in range(N_mcs)])
np.unique(Tmax)

array([0.])

In [13]:
plt.plot(toocan[0].clusters.Utime,toocan[0].clusters.surfkm2_132Wm2)

In [14]:
toocan[0].clusters.Utime[0], toocan[0].clusters.Utime[-1]

(17014.18, 17014.42)